# ERUPT under simulated random assignment

In [12]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))

from causaltune import CausalTune
from causaltune.datasets import generate_non_random_dataset
from causaltune.erupt import DummyPropensity, ERUPT


In [2]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
%%javascript

// turn off scrollable windows for large output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Loading data and model training

In [4]:
# load toy dataset with non-random assignment and apply standard pre-processing
cd = generate_non_random_dataset()
cd.preprocess_dataset()

In [5]:
display(cd.data.head())

,T,Y,random,X1,X2,X3,X4,X5,propensity
0,0,-0.529094,0.0,-0.325404,-3.200259,-1.096231,0.454945,-0.682950,0.096673
1,0,-2.673912,1.0,-2.224641,1.384133,0.506485,0.145684,-0.195266,0.472952
2,1,-1.666444,0.0,0.687121,-0.207614,0.788699,1.131345,-0.352091,0.550413
3,0,-1.619143,0.0,0.740413,-0.666263,1.027818,-0.197965,-2.025220,0.423549
4,0,0.331106,1.0,-0.907719,-1.775581,0.072270,-1.760379,1.449668,0.083704


In [6]:
# training configs

# set evaluation metric
metric = "energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
time_budget = None
components_time_budget = 10

# specify training set size
train_size = 0.7

Now if `outcome_model="auto"` in the CausalTune constructor, we search over a simultaneous search space for the EconML estimators and for FLAML wrappers for common regressors. The old behavior is now achieved by `outcome_model="nested"` (Refitting AutoML for each estimator).

You can also preprocess the data in the CausalityDataset using one of the popular category encoders: OneHot, WoE, Label, Target.

In [7]:
ct = CausalTune(
    estimator_list=["CausalForestDML", "XLearner"],
    metric=metric,
    verbose=0,
    components_verbose=0,
    time_budget=time_budget,
    components_time_budget=components_time_budget,
    train_size=train_size,
    outcome_model="auto"
)


# run causaltune
ct.fit(data=cd, outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct.best_estimator}")
# config of best estimator:
print(f"Best config: {ct.best_config}")
# best score:
print(f"Best score: {ct.best_score}")

Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Propensity Model Fitted Successfully
---------------------
Best estimator: backdoor.econml.dml.CausalForestDML
Best config: {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': 1, 'n_estimators': 2, 'criterion': 'het', 'min_samples_split': 12, 'min_samples_leaf': 8, 'min_weight_fraction_leaf': 0.0, 'max_features': 'log2', 'min_impurity_decrease': 0, 'max_samples': 0.2884902061383809, 'min_balancedness_tol': 0.4585520111743354, 'honest': 1, 'fit_intercept': 1, 'subforest_size': 5}, 'outcome_estimator': {'alpha': 0.006205274971406812, 'fit_intercept': True, 'eps': 7.833744321548246e-15, 'estimator_name': 'lasso_lars'}}
Best score: 0.2952285030581425


## Random ERUPT

Below we demonstrate how to use Estimated Response Under Proposed Treatment (ERUPT) to estimate the average treatment effect had the treatment been assigned randomly. Recall that the dataset used in this example is constructed in a way that the treatment propensity is a function of a unit's covariates.

In [8]:
use_df = ct.test_df

In [9]:
# computing mean ERUPT over 10 bootstrapped samples

scores_list = []

for i in range(10):

    bootstrap_df = use_df.sample(frac=1, replace=True)
    propensities = bootstrap_df['propensity']
    actual_treatment = bootstrap_df['T']
    outcome = bootstrap_df['Y']

    # define the random assignment policy
    random_policy = np.random.randint(0,2, size=len(bootstrap_df))

    # define a propensity model that will simply return the propensities when calling predict_proba
    propensity_model = DummyPropensity(p=propensities, treatment=actual_treatment)

    # obtain ERUPT under random policy
    e = ERUPT(treatment_name='T', propensity_model=propensity_model)
    scores_list.append(e.score(df=use_df,outcome=outcome,policy=random_policy))

erupt_mean = np.mean(scores_list)
erupt_sd = np.std(scores_list)

In [10]:
# compute naive ate as difference in means
naive_ate, naive_sd, _ = ct.scorer.naive_ate(ct.test_df['T'], ct.test_df['Y'])

In [11]:
# comparison of naive ate to mean random erupt over 10 bootstrap runs
erupt_df = pd.DataFrame([[naive_ate,naive_sd],[erupt_mean,erupt_sd]], columns=['estimated_effect', 'sd'], index=['naive_ate','random_erupt'])
display(erupt_df)

,estimated_effect,sd
naive_ate,0.030740,0.139801
random_erupt,-0.001059,0.210618


For more details on the ERUPT implementation, consult [Hitsch and Misra (2018)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957). Note also that we assume that treatment takes integer values from 0 to n.